### Examples Of Pyspark ML

In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()
# Setting default log level to "WARN".
# To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
# 22/07/30 23:34:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
# 22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
# 22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
# 22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
# 22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
# 22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/30 23:34:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/07/30 23:34:15 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [2]:
## Read The dataset
training = spark.read.csv('test1.csv',header=True,inferSchema=True)

In [3]:
training.show()



+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [6]:
[Age,Experience]----> new feature--->independent feature

SyntaxError: invalid syntax (1054366730.py, line 1)

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [8]:
output=featureassembler.transform(training)

In [9]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [10]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [11]:
finalized_data=output.select("Independent Features","Salary")

In [12]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [13]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)
# 22/07/30 23:36:25 WARN Instrumentation: [8b4e8f07] regParam is zero, which might cause numerical instability and overfitting.
# 22/07/30 23:36:26 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS

22/07/30 23:36:25 WARN Instrumentation: [8b4e8f07] regParam is zero, which might cause numerical instability and overfitting.
22/07/30 23:36:26 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [14]:
### Coefficients
regressor.coefficients

DenseVector([-250.0, 1750.0])

In [15]:
### Intercepts
regressor.intercept

20250.000000000553

In [16]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [17]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [21.0,1.0]| 15000| 16750.00000000008|
|          [24.0,3.0]| 20000|19500.000000000044|
|          [30.0,8.0]| 25000|26749.999999999993|
+--------------------+------+------------------+



In [18]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1333.333333333343, 2125000.0000000703)